In [1]:
# awalbp.ipynb - PuLP implementation of AWALBP-L2-1 due to [Fleszar 2017] https://doi.org/10.1016/j.ejor.2016.10.008
# Copyright (c) 2025 Benjamin Futász
#
# This software is provided 'as-is', without any express or implied
# warranty. In no event will the authors be held liable for any damages
# arising from the use of this software.
#
# Permission is granted to anyone to use this software for any purpose,
# including commercial applications, and to alter it and redistribute it
# freely, subject to the following restrictions:
#
# 1. The origin of this software must not be misrepresented; you must not
#    claim that you wrote the original software. If you use this software
#    in a product, an acknowledgment in the product documentation would be
#    appreciated but is not required.
# 2. Altered source versions must be plainly marked as such, and must not be
#    misrepresented as being the original software.
# 3. This notice may not be removed or altered from any source distribution.

In [2]:
import pathlib

import pulp
import numpy as np

import parse  # parse.py (zlib License)

In [3]:
"""
The parameter set was introduced in the following paper.
We gratefully received the files from the original authors after requesting them by e-mail.

@article{calleja2013milp,
    title={A MILP model for the accessibility windows assembly line balancing problem (AWALBP)},
    author={Calleja, Gema and Corominas, Albert and Garc{\'\\i}a-Villoria, Alberto and Pastor, Rafael},
    journal={International Journal of Production Research},
    volume={51},
    number={12},
    pages={3549--3560},
    year={2013},
    publisher={Taylor \\& Francis}
}
https://doi.org/10.1080/00207543.2012.751514
"""

# parameters
SCRIPT_PATH = pathlib.Path(".").parent.resolve()
folder = "test_instances"
pex = "A_12_m_5_N_50_1.txt"

# workload
fp = SCRIPT_PATH / folder / pex
data = parse.convert_to_dict_and_validate(fp, parse.names, verbose=False)
for key, value in data.items():
    print(f"{key}: {value}, len: {len(value)}")

number_of_tasks: [83], len: 1
number_of_workstations: [7], len: 1
accessibility_window_L: [0, 11, 22, 33, 44, 55, 66], len: 7
accessibility_window_R: [10, 21, 32, 43, 54, 65, 76], len: 7
size_of_workpiece_horizontal: [14], len: 1
distance_between_right_borders: [15], len: 1
intermediate_time_between_stages: [200], len: 1
elementary_step_length: [1], len: 1
tasks: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83], len: 83
workstation_tasks_1: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14], len: 14
workstation_tasks_2: [15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28], len: 14
workstation_tasks_3: [29, 30, 31, 32, 33, 34, 35], len: 7
workstation_tasks_4: [36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48], len

In [4]:
# parameter names per Fleszar 2017 AWALBP-L2-1
n = data["number_of_tasks"][0]
m = data["number_of_workstations"][0]

p = data["task_processing_time"]
a = data["distance_to_right_border_of_workpiece"]  # defines component position on workpiece

L = data["accessibility_window_L"]
R = data["accessibility_window_R"]
J = [np.array(data[f"workstation_tasks_{i+1}"]) - 1 for i in range(m)]  # note conversion to 0-based indexes

A = data["distance_between_right_borders"][0]
T = data["intermediate_time_between_stages"][0]

#

tasks = np.arange(n)
wrksts = np.arange(m)
stages = np.arange(A)



In [5]:
class AWALBP_L2_1_Fleszar2017():
    """
    Source:

    @article{fleszar2017new,
        title={A new MILP model for the accessibility windows assembly line balancing problem level 2 (AWALBP-L2)},
        author={Fleszar, Krzysztof},
        journal={European journal of operational research},
        volume={259},
        number={1},
        pages={169--174},
        year={2017},
        publisher={Elsevier}
    }

    https://doi.org/10.1016/j.ejor.2016.10.008
    """
    def __init__(self):
        # Pre-processing
        def task_accessible(j,i,s):
            if j in J[i]:
                for k in range(max(R)):
                    tmp = s+1 + A*k - a[j]
                    if L[i] <= tmp <= R[i]:
                        return True
            return False

        E = [(j,s) for j in range(n) for i in range(m) for s in range(A) if task_accessible(j,i,s)]
        E.sort()

        # Define problem
        model = pulp.LpProblem("AWALBP-L2-1_Fleszar2017", pulp.LpMinimize)
        y = pulp.LpVariable.dicts("y", (tasks,stages), cat="Binary")
        x = pulp.LpVariable.dicts("x", (stages), cat="Binary")
        c = pulp.LpVariable.dicts("c", (stages), lowBound=0.0)

        # Objective function
        model += pulp.lpSum(
            c[s] + T * x[r] for s in stages for r in stages
        ), "Total_Costs"

        # Constraints
        for i in wrksts:
            for s in stages:
                model += pulp.lpSum(p[j]*y[j][s] for j in J[i] if (j,s) in E) <= c[s], f"cons_sum_py<=c_{i}_{s}"
        for j in tasks:
            model += pulp.lpSum(y[j][s] for s in stages if (j,s) in E) >= 1.0, f"cons_sum_y>=1_{j}"
        for j,s in E:
            model += y[j][s] <= x[s], f"cons_y<=x_{j}_{s}"

        self.model = model

    def solve(self, solver=pulp.CUOPT(msg=1)):
        self.model.solve(solver)

        self.status = self.model.status
        self.objective = self.model.objective

    def variables(self):
        return self.model.variables()

In [6]:
model = AWALBP_L2_1_Fleszar2017()

In [7]:
model.solve(pulp.CUOPT(msg=1))

Setting parameter infeasibility_detection to true
Setting parameter log_to_console to true
Setting parameter log_file to 
Solving a problem with 1101 constraints 943 variables (928 integers) and 3757 nonzeros
Objective offset 0.000000 scaling_factor 1.000000
After trivial presolve updated number of 1101 constraints 943 variables
Running presolve!
Solving LP root relaxation
Scaling matrix. Maximum column norm 5.787918e+00
Dual Simplex Phase 1
Dual feasible solution found.
Dual Simplex Phase 2
 Iter     Objective   Primal Infeas  Perturb  Time
    1 +0.00000000e+00 6.70613933e+02 0.00e+00 0.01
 1000 +1.93490139e+04 9.28217820e+01 0.00e+00 0.10
 2000 +2.94450000e+04 7.27016959e+02 0.00e+00 0.18
Removed perturbation of 1.63e-05.

Root relaxation solution found in 2547 iterations and 0.24s
Root relaxation objective +3.10567891e+04

Strong branching using 2 threads and 173 fractional variables
New solution from primal heuristics. Objective +5.366948e+04. Time 0.52
New solution from primal he

In [8]:
print("Status:", pulp.LpStatus[model.status])
print("Total Costs =", pulp.value(model.objective))

Status: Optimal
Total Costs = 32504.999999999993


In [9]:
model.model

AWALBP-L2-1_Fleszar2017:
MINIMIZE
15*c_0 + 15*c_1 + 15*c_10 + 15*c_11 + 15*c_12 + 15*c_13 + 15*c_14 + 15*c_2 + 15*c_3 + 15*c_4 + 15*c_5 + 15*c_6 + 15*c_7 + 15*c_8 + 15*c_9 + 3000*x_0 + 3000*x_1 + 3000*x_10 + 3000*x_11 + 3000*x_12 + 3000*x_13 + 3000*x_14 + 3000*x_2 + 3000*x_3 + 3000*x_4 + 3000*x_5 + 3000*x_6 + 3000*x_7 + 3000*x_8 + 3000*x_9 + 0.0
SUBJECT TO
cons_sum_py<=c_0_0: - c_0 + 121 y_0_0 + 134 y_10_0 + 139 y_11_0 + 119 y_12_0
 + 145 y_13_0 + 143 y_3_0 + 112 y_4_0 + 125 y_6_0 + 125 y_7_0 <= 0

cons_sum_py<=c_0_1: - c_1 + 134 y_10_1 + 139 y_11_1 + 119 y_12_1 + 145 y_13_1
 + 143 y_3_1 + 112 y_4_1 + 125 y_6_1 + 125 y_7_1 <= 0

cons_sum_py<=c_0_2: - c_2 + 139 y_11_2 + 119 y_12_2 + 143 y_3_2 + 125 y_6_2
 + 125 y_7_2 + 128 y_8_2 <= 0

cons_sum_py<=c_0_3: - c_3 + 119 y_12_3 + 107 y_1_3 + 143 y_3_3 + 130 y_5_3
 + 125 y_6_3 + 125 y_7_3 + 128 y_8_3 <= 0

cons_sum_py<=c_0_4: - c_4 + 107 y_1_4 + 119 y_2_4 + 143 y_3_4 + 130 y_5_4
 + 125 y_6_4 + 125 y_7_4 + 128 y_8_4 + 120 y_9_4 <= 0

cons_sum_